In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from pylab import mpl
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.pipeline import Pipeline
import seaborn as sns

In [2]:
test3 = pd.read_csv('all_sheets_merge_T1 - all_sheets_merge_0430.csv')

In [3]:
test3=test3.fillna(0)

In [4]:
from sklearn.decomposition import PCA
import re
label_df = test3['呼吸器成功脫離']
test3['性別'] = test3['性別'].map({'M':0,'F':1})
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X = test3.drop(['呼吸器成功脫離'], axis=1)
pca = PCA(n_components = 5)
pca_vectors = pca.fit_transform(X)
y = test3['呼吸器成功脫離']


In [ ]:
test3

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_df = scaler.fit_transform(X)
pca = PCA(n_components = 5)
pca_vectors = pca.fit_transform(scaled_df)
for index, var in enumerate(pca.explained_variance_ratio_): print("Explained Variance ratio by Principal Component ", (index+1), " : ", var)


In [ ]:
plt.figure(figsize=(25,8)) 
sns.scatterplot(x=pca_vectors[:, 0], 
                y=pca_vectors[:, 1], hue=label_df) 
plt.title('Principal Components vs Class distribution', fontsize=16) 
plt.ylabel('Principal Component 2', fontsize=16) 
plt.xlabel('Principal Component 1', fontsize=16) 
plt.xticks(rotation='vertical')

In [12]:
X_train, X_test, y_train, y_test = train_test_split(pca_vectors, y, test_size=0.3, random_state=33)
X_train

array([[-2.02387496e+04, -1.74335212e+04,  7.63234293e+03,
         1.54767385e+03,  2.34211015e+03],
       [ 1.24766806e+04, -1.18511859e+04,  5.42037208e+03,
         6.32300577e+03, -8.71353446e+03],
       [-2.02346576e+04, -9.48539334e+03,  1.12245462e+03,
         6.14179239e+03,  8.91246303e+03],
       [ 2.42413298e+04, -1.39162334e+04, -8.91472574e+03,
        -1.09629910e+04,  5.35939975e+03],
       [ 3.79459274e+03,  7.64933038e+03, -6.18270624e+03,
        -8.18848627e+03,  5.84481565e+03],
       [-1.72481652e+04, -1.60282052e+04,  1.36795690e+04,
         2.57561988e+03,  1.63799461e+03],
       [-1.03614090e+04, -1.09120106e+04,  1.29791525e+04,
        -1.46044306e+04, -1.64967353e+03],
       [-1.24322144e+04, -6.99734571e+03,  7.08137747e+03,
        -1.19697105e+04, -5.30435346e+03],
       [ 1.52830294e+04, -8.42938332e+03, -8.32563241e+03,
        -8.58549659e+03,  3.21816512e+03],
       [-8.84232060e+03,  7.60617467e+03,  1.79833270e+03,
         4.72871951e+03

In [18]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5) 
knn.fit(X_train,y_train)
scores = cross_val_score(knn,X,y,cv=10,scoring='accuracy')
print(scores)
print(scores.mean())

[0.59090909 0.59090909 0.5        0.63636364 0.36363636 0.54545455
 0.63636364 0.72727273 0.63636364 0.59090909]
0.5818181818181819


In [19]:
k_range = range(1,55)
k_scores = []
for k_number in k_range:
    knn = KNeighborsClassifier(n_neighbors=k_number)
    scores = cross_val_score(knn,X,y,cv=5,scoring='accuracy')
    k_scores.append(scores.mean())

In [ ]:
plt.plot(k_range,k_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')
plt.show()

In [ ]:
xgbc = XGBClassifier()

In [ ]:
from sklearn.linear_model import LogisticRegression

pl_log_reg_pca = Pipeline(steps=[('scaler',StandardScaler()), ('pca', PCA(n_components = 5)), ('log_reg', LogisticRegression(multi_class='multinomial', solver='saga', max_iter=200))])
scores = cross_val_score(pl_log_reg_pca, test3, label_df, cv=10,scoring='accuracy')
print('Accuracy for Logistic Regression with 5 Principal Components: ', scores.mean())

In [ ]:
test3=test3.fillna(0)

In [ ]:
import xgboost as xgb
pl_xgb = Pipeline(steps= [('xgboost', xgb.XGBClassifier(objective='multi:softmax'))]) 
scores = cross_val_score(pl_xgb, test3, label_df, cv=10) 
print('Accuracy for XGBoost Classifier : ', scores.mean())

In [ ]:
X = test3[['Age', 'RCC_APACHEII', '氣切', 'Difference APACHEII 6', 'Difference APACHEII 8', 'Difference APACHEII 9', '呼吸器天數', 'RCC天數', '插管-氣切',]]

In [ ]:
y = test3['呼吸器成功脫離']

In [ ]:
test3 = test3.drop(labels=['呼吸器成功脫離'],axis='columns')

In [ ]:
X = test3

In [ ]:
y = test3['呼吸器成功脫離']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)
X_train

In [13]:
xgbc = XGBClassifier()

In [14]:
xgbc.fit(X_train, y_train)
print('The accuracy of eXtreme Gradient Boosting Classifier on testing set:', xgbc.score(X_test, y_test))

The accuracy of eXtreme Gradient Boosting Classifier on testing set: 0.6515151515151515


In [ ]:
from xgboost import plot_importance
plot_importance(xgbc)
plt.show()

In [ ]:
print('The accuracy of eXtreme Gradient Boosting Classifier on testing set:', xgbc.score(X_test, y_test))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

In [ ]:
X_train

In [ ]:
xgbc.fit(X_train, y_train)

In [ ]:
pd.set_option("display.max_rows", 1000)

In [ ]:
X_train

In [ ]:
test

In [ ]:
test.drop(0)

In [ ]:
X_train

In [ ]:
test

In [ ]:
test.drop(0)

In [ ]:
test

In [ ]:
test = test.drop(0)

In [ ]:
test

In [ ]:
test

In [ ]:
X_train

In [ ]:
X = test[['基本資料', 'Unnamed: 7', 'Unnamed: 9']]
y = test['Unnamed: 19']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

In [ ]:
X_train

In [ ]:
xgbc.fit(X_train, y_train)

In [ ]:
X = test[['Unnamed: 7', 'Unnamed: 9']]
y = test['Unnamed: 19']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

In [ ]:
X_train

In [ ]:
xgbc = XGBClassifier()

In [ ]:
Unnamed: 7 = int()

In [ ]:
xgbc.fit(X_train, y_train)

In [ ]:
X = test[['Unnamed: 7', 'Unnamed: 9']]
y = test['Unnamed: 19']

In [ ]:
X_train

In [ ]:
xgbc = XGBClassifier()

In [ ]:
test